In [1]:
import pandas as pd
from util import *
pd.set_option('display.float_format', lambda x: "{:,}".format(x))

## Dataset Description and Notes

- understanding the type of population of each county (households, relationships, industry, education levels, etc)
- which types contribute to the spread most and which have managed the spread best?
- may be useful in deriving commonalities in counties in order to develop nuanced intervention approaches in different areas
- population estimates for counties w/ 90% confindence level

## Reading in datasets

In [ ]:
# read data set
economic = pd.read_csv("../../data/acs/economic/economic.csv", header=1)
social = pd.read_csv("../../data/acs/social/social.csv", header=1)
# social_pr = pd.read_csv("../../data/acs/social_pr.csv")
overall = pd.read_csv("../../data/superCOVID-19dataframe.csv", index_col = "countyFIPS")
overall.drop(0, inplace=True)

## Calculating State Trends Over Past Week - Georgia

In [ ]:
ga = get_state(overall, 'GA')

In [4]:
ga_c = get_cases(ga, days_back = 7)
ga_d = get_deaths(ga, days_back = 7)

In [5]:
ga_c.head()

,confirmed_11/6/20,confirmed_11/7/20,confirmed_11/8/20,confirmed_11/9/20,confirmed_11/10/20,confirmed_11/11/20,confirmed_11/12/20
countyFIPS,,,,,,,
13001,3.0,1.0,4.0,1.0,0.0,0.0,6.0
13003,0.0,0.0,1.0,1.0,1.0,1.0,3.0
13005,2.0,3.0,1.0,1.0,0.0,0.0,1.0
13007,0.0,0.0,2.0,1.0,0.0,0.0,2.0
13009,7.0,9.0,2.0,13.0,10.0,6.0,3.0


In [6]:
ga_d.head()

,deaths_11/6/20,deaths_11/7/20,deaths_11/8/20,deaths_11/9/20,deaths_11/10/20,deaths_11/11/20,deaths_11/12/20
countyFIPS,,,,,,,
13001,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13003,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13005,0.0,1.0,0.0,0.0,0.0,1.0,0.0
13007,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13009,0.0,0.0,0.0,1.0,0.0,0.0,1.0


### During the week in question, the number of confirmed cases and deaths in the state of Georgia were both determined to be increasing

## Manipulating "id" column to match "countyFIPS" in main dataset for merging

In [7]:
economic['id'] = economic['id'].str.replace("0500000US", "")
social['id'] = social['id'].str.replace("0500000US", "")

economic['id'] = economic['id'].astype('int64')
social['id'] = social['id'].astype('int64')

economic.rename(columns = {"id": "countyFIPS"}, inplace=True)
social.rename(columns = {"id": "countyFIPS"}, inplace=True)
economic.set_index("countyFIPS", inplace=True)
social.set_index("countyFIPS", inplace=True)

# Removing "Margin of Error" columns
economic = trim_cols(economic)
social = trim_cols(social)

social[["County Name", "State"]] = social["Geographic Area Name"].str.split(", ", expand=True)
economic[["County Name", "State"]] = economic["Geographic Area Name"].str.split(", ", expand=True)

social.drop(columns=['Geographic Area Name'], inplace=True)
economic.drop(columns=['Geographic Area Name'], inplace=True)

economic.to_csv("econ.csv")
social.to_csv("soc.csv")

In [8]:
social.head()

,Estimate!!HOUSEHOLDS BY TYPE!!Total households,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households,Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families),Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families),Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!With own children of the householder under 18 years,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!With own children of the householder under 18 years,Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Married-couple family,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Married-couple family,Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Married-couple family!!With own children of the householder under 18 years,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Married-couple family!!With own children of the householder under 18 years,...,Estimate!!ANCESTRY!!Total population!!West Indian (excluding Hispanic origin groups),Percent Estimate!!ANCESTRY!!Total population!!West Indian (excluding Hispanic origin groups),Estimate!!COMPUTERS AND INTERNET USE!!Total households,Percent Estimate!!COMPUTERS AND INTERNET USE!!Total households,Estimate!!COMPUTERS AND INTERNET USE!!Total households!!With a computer,Percent Estimate!!COMPUTERS AND INTERNET USE!!Total households!!With a computer,Estimate!!COMPUTERS AND INTERNET USE!!Total households!!With a broadband Internet subscription,Percent Estimate!!COMPUTERS AND INTERNET USE!!Total households!!With a broadband Internet subscription,County Name,State
countyFIPS,,,,,,,,,,,,,,,,,,,,,
1001,"21,115.0","21,115.0","15,161.0",71.8,"6,787.0",32.1,"11,988.0",56.8,"5,201.0",24.6,...,0.0,0.0,"21,115.0","21,115.0","18,368.0",87.0,"16,651.0",78.9,Autauga County,Alabama
1003,"78,622.0","78,622.0","51,359.0",65.3,"18,035.0",22.9,"41,452.0",52.7,"13,527.0",17.2,...,864.0,0.4,"78,622.0","78,622.0","69,635.0",88.6,"61,424.0",78.1,Baldwin County,Alabama
1005,"9,186.0","9,186.0","6,030.0",65.6,"2,423.0",26.4,"3,908.0",42.5,"1,249.0",13.6,...,33.0,0.1,"9,186.0","9,186.0","6,986.0",76.1,"5,548.0",60.4,Barbour County,Alabama
1007,"6,840.0","6,840.0","4,947.0",72.3,"1,728.0",25.3,"3,626.0",53.0,"1,321.0",19.3,...,0.0,0.0,"6,840.0","6,840.0","5,217.0",76.3,"4,521.0",66.1,Bibb County,Alabama
1009,"20,600.0","20,600.0","15,104.0",73.3,"5,566.0",27.0,"11,942.0",58.0,"4,129.0",20.0,...,13.0,0.0,"20,600.0","20,600.0","16,202.0",78.7,"14,103.0",68.5,Blount County,Alabama


## Merging with super dataset

In [9]:
merged = social.merge(economic, how="outer", left_index=True, right_index=True)

In [10]:
# merged.to_csv("../../data/jamison/social_and_economic.csv")

In [11]:
# merged = merged.merge(overall, left_index=True, right_index=True)

## Final Result

In [12]:
merged.head()

,Estimate!!HOUSEHOLDS BY TYPE!!Total households,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households,Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families),Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families),Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!With own children of the householder under 18 years,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!With own children of the householder under 18 years,Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Married-couple family,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Married-couple family,Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Married-couple family!!With own children of the householder under 18 years,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Married-couple family!!With own children of the householder under 18 years,...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!18 years and over!!18 to 64 years,Percent Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!18 years and over!!18 to 64 years,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!18 years and over!!65 years and over,Percent Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!18 years and over!!65 years and over,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!People in families,Percent Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!People in families,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!Unrelated individuals 15 years and over,Percent Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!Unrelated individuals 15 years and over,County Name_y,State_y
countyFIPS,,,,,,,,,,,,,,,,,,,,,
1001,"21,115.0","21,115.0","15,161.0",71.8,"6,787.0",32.1,"11,988.0",56.8,"5,201.0",24.6,...,(X),14.0,(X),8.7,(X),14.0,(X),24.3,Autauga County,Alabama
1003,"78,622.0","78,622.0","51,359.0",65.3,"18,035.0",22.9,"41,452.0",52.7,"13,527.0",17.2,...,(X),10.4,(X),8.0,(X),7.9,(X),23.2,Baldwin County,Alabama
1005,"9,186.0","9,186.0","6,030.0",65.6,"2,423.0",26.4,"3,908.0",42.5,"1,249.0",13.6,...,(X),25.4,(X),16.2,(X),27.8,(X),34.4,Barbour County,Alabama
1007,"6,840.0","6,840.0","4,947.0",72.3,"1,728.0",25.3,"3,626.0",53.0,"1,321.0",19.3,...,(X),13.3,(X),8.1,(X),11.3,(X),31.9,Bibb County,Alabama
1009,"20,600.0","20,600.0","15,104.0",73.3,"5,566.0",27.0,"11,942.0",58.0,"4,129.0",20.0,...,(X),12.9,(X),10.1,(X),12.2,(X),30.4,Blount County,Alabama


## Exporting

In [13]:
# merged.to_csv("../../data/jamison/social_and_economic.csv")

## Renaming columns

In [14]:
# cols = [x.replace("Estimate!!", "est_") for x in merged.columns]
# # print(cols[0:20])
# cols = [x.replace("HOUSEHOLDS BY TYPE!!", "households_") for x in cols]
# # print(cols[0:20])
# cols = [x.replace("Total households!!", "total_") for x in cols]
# # print(cols[0:20])
# cols = [x.replace("Family households (families)!!", "families_") for x in cols]
# # print(cols[0:20])
# cols = [x.replace("Married-couple family", "married_") for x in cols]
# # print(cols[0:20])

# merged.columns = cols
# merged

## VARIABLES


In [15]:
households = merged[list(filter(lambda x: "HOUSEHOLDS" in x, merged.columns))]
relationship = merged[list(filter(lambda x: "RELATIONSHIP" in x, merged.columns))]
marital = merged[list(filter(lambda x: "MARITAL" in x, merged.columns))]
fertility = merged[list(filter(lambda x: "FERTILITY" in x, merged.columns))]
grandparents = merged[list(filter(lambda x: "GRANDPARENTS" in x, merged.columns))]
in_school = merged[list(filter(lambda x: "SCHOOL" in x, merged.columns))]
education_level = merged[list(filter(lambda x: "EDUCATION" in x, merged.columns))]
veteran = merged[list(filter(lambda x: "VETERAN" in x, merged.columns))]
disability = merged[list(filter(lambda x: "DISABILITY" in x, merged.columns))]
birth = merged[list(filter(lambda x: "BIRTH" in x, merged.columns))]
citizenship = merged[list(filter(lambda x: "CITIZENSHIP" in x, merged.columns))]
home_lang = merged[list(filter(lambda x: "LANGUAGE" in x, merged.columns))]
ancestry = merged[list(filter(lambda x: "ANCESTRY" in x, merged.columns))]
computer = merged[list(filter(lambda x: "COMPUTER" in x, merged.columns))]

variables = [households, relationship, marital, fertility, grandparents, in_school, education_level, veteran, disability, birth, citizenship, home_lang, ancestry, computer]

In [24]:
employment = merged[list(filter(lambda x: "EMPLOYMENT" in x, merged.columns))]
commuting = merged[list(filter(lambda x: "COMMUTING" in x, merged.columns))]
occupation = merged[list(filter(lambda x: "OCCUPATION" in x, merged.columns))]
industry = merged[list(filter(lambda x: "INDUSTRY" in x, merged.columns))]
work_class = merged[list(filter(lambda x: "WORKER" in x, merged.columns))]
income = merged[list(filter(lambda x: "INCOME" in x, merged.columns))]
insurance = merged[list(filter(lambda x: "INSURANCE" in x, merged.columns))]
poverty = merged[list(filter(lambda x: "POVERTY" in x, merged.columns))]


variables2 = [employment, commuting, occupation, industry, work_class, income, insurance, poverty]

In [26]:
for col in commuting.columns:
    print(col)

Estimate!!COMMUTING TO WORK!!Workers 16 years and over
Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van -- drove alone
Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van -- drove alone
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van -- carpooled
Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van -- carpooled
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Public transportation (excluding taxicab)
Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Public transportation (excluding taxicab)
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Walked
Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Walked
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Other means
Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Other means
Estimate!!COMMUTING TO WORK!

In [22]:
# employment.to_csv("../../data/jamison/variables/employment.csv")
# commuting.to_csv("../../data/jamison/variables/commuting.csv")
# occupation.to_csv("../../data/jamison/variables/occupation.csv")
# industry.to_csv("../../data/jamison/variables/industry.csv")
# work_class.to_csv("../../data/jamison/variables/work_class.csv")
# income.to_csv("../../data/jamison/variables/income.csv")
# insurance.to_csv("../../data/jamison/variables/insurance.csv")
# poverty.to_csv("../../data/jamison/variables/poverty.csv")

In [16]:
# households.to_csv("../../data/jamison/variables/households.csv")
# relationship.to_csv("../../data/jamison/variables/relationship.csv")
# marital.to_csv("../../data/jamison/variables/marital.csv")
# fertility.to_csv("../../data/jamison/variables/fertility.csv")
# grandparents.to_csv("../../data/jamison/variables/grandparents.csv")
# in_school.to_csv("../../data/jamison/variables/in_school.csv")
# education_level.to_csv("../../data/jamison/variables/education_level.csv")
# veteran.to_csv("../../data/jamison/variables/veteran.csv")
# disability.to_csv("../../data/jamison/variables/disability.csv")
# birth.to_csv("../../data/jamison/variables/birth.csv")
# citizenship.to_csv("../../data/jamison/variables/citizenship.csv")
# home_lang.to_csv("../../data/jamison/variables/home_lang.csv")
# ancestry.to_csv("../../data/jamison/variables/ancestry.csv")
# computer.to_csv("../../data/jamison/variables/computer.csv")

In [19]:
grandparents

,Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years,Percent Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years,Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Grandparents responsible for grandchildren,Percent Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Grandparents responsible for grandchildren,Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!Less than 1 year,Percent Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!Less than 1 year,Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!1 or 2 years,Percent Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!1 or 2 years,Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!3 or 4 years,Percent Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!3 or 4 years,Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!5 or more years,Percent Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!5 or more years,Estimate!!GRANDPARENTS!!Number of grandparents responsible for own grandchildren under 18 years,Percent Estimate!!GRANDPARENTS!!Number of grandparents responsible for own grandchildren under 18 years,Estimate!!GRANDPARENTS!!Number of grandparents responsible for own grandchildren under 18 years!!Who are female,Percent Estimate!!GRANDPARENTS!!Number of grandparents responsible for own grandchildren under 18 years!!Who are female,Estimate!!GRANDPARENTS!!Number of grandparents responsible for own grandchildren under 18 years!!Who are married,Percent Estimate!!GRANDPARENTS!!Number of grandparents responsible for own grandchildren under 18 years!!Who are married
countyFIPS,,,,,,,,,,,,,,,,,,
1001,"1,109.0","1,109.0",670.0,60.4,98.0,8.8,184.0,16.6,68.0,6.1,320.0,28.9,670.0,670.0,398.0,59.4,485.0,72.4
1003,"4,077.0","4,077.0","2,084.0",51.1,473.0,11.6,299.0,7.3,445.0,10.9,867.0,21.3,"2,084.0","2,084.0","1,265.0",60.7,"1,473.0",70.7
1005,766.0,766.0,361.0,47.1,55.0,7.2,43.0,5.6,110.0,14.4,153.0,20.0,361.0,361.0,261.0,72.3,232.0,64.3
1007,724.0,724.0,545.0,75.3,169.0,23.3,18.0,2.5,134.0,18.5,224.0,30.9,545.0,545.0,333.0,61.1,426.0,78.2
1009,"1,642.0","1,642.0",882.0,53.7,141.0,8.6,71.0,4.3,187.0,11.4,483.0,29.4,882.0,882.0,496.0,56.2,787.0,89.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72145,nan,nan,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,nan,nan,NaN,nan,NaN
72147,nan,nan,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,nan,nan,NaN,nan,NaN
72149,nan,nan,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,nan,nan,NaN,nan,NaN


In [23]:
commuting.columns


Index(['Estimate!!COMMUTING TO WORK!!Workers 16 years and over',
       'Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over',
       'Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van -- drove alone',
       'Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van -- drove alone',
       'Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van -- carpooled',
       'Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van -- carpooled',
       'Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Public transportation (excluding taxicab)',
       'Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Public transportation (excluding taxicab)',
       'Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Walked',
       'Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Walked',
       'Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Other me

In [18]:
p = get_percents(households)
p

,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families),Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!With own children of the householder under 18 years,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Married-couple family,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Married-couple family!!With own children of the householder under 18 years,"Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Male householder, no wife present, family","Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Male householder, no wife present, family!!With own children of the householder under 18 years","Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Female householder, no husband present, family","Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Female householder, no husband present, family!!With own children of the householder under 18 years",Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Nonfamily households,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Nonfamily households!!Householder living alone,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Nonfamily households!!Householder living alone!!65 years and over,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Households with one or more people under 18 years,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Households with one or more people 65 years and over,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Average household size,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Average family size
countyFIPS,,,,,,,,,,,,,,,
1001,71.8,32.1,56.8,24.6,3.6,1.8,11.4,5.7,28.2,25.1,10.5,36.1,27.1,(X),(X)
1003,65.3,22.9,52.7,17.2,3.1,1.4,9.5,4.3,34.7,30.4,12.8,26.2,35.0,(X),(X)
1005,65.6,26.4,42.5,13.6,3.8,1.8,19.3,10.9,34.4,32.5,14.5,31.7,36.2,(X),(X)
1007,72.3,25.3,53.0,19.3,5.7,1.2,13.7,4.8,27.7,26.0,12.9,29.2,35.4,(X),(X)
1009,73.3,27.0,58.0,20.0,5.8,2.4,9.6,4.6,26.7,24.7,11.0,31.2,32.3,(X),(X)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72145,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN
72147,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN
72149,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN


## Challenges

- how to rename columns and how to do so efficiently/programmatically
- 1000+
- many different variable groups, many more subgroups
- finding which columns or most relevant and which can be removed w/o losing critical info
- which pandas methods work best for certain purposes
- how to arrange and group data
- displaying numbers in readable format (not scientific notation)

## Considerations
- Break dataframes down in layers starting w/ highest order / root variable?
    - This would produce almost 20 dataframe which would then be broken down further
- Remove percent estimate columns (may be extracted by leveraging total estimates)
    - This would cut the number of columns by 50%

## Issues
- total househoulds estimates and percent estimates are the same, what went wrong?